<img src="../../img/python-logo-no-text.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;">
  <b>Case Study: Othellite</b>
</div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>
<br/>
<div style="text-align:center;">module_220_advanced_topics/topic_900_othellite</div>



In [ ]:
from collections.abc import Sequence
from enum import Enum
from dataclasses import dataclass, field
import reprlib
from typing import Union
from random import sample

try:
    import numpy as np
except ModuleNotFoundError:
    print("NumPy not found, some (minor) examples may not work")

# Othellite

In this notebook we want to implement a simplified variant of the game Reversi (also
known under the trade name Othello). The game is
played on a board with 8x8 squares, on which players
can place black or white pieces. The exact rules are
described [in the Wikipedia article](https://en.wikipedia.org/wiki/Reversi).

We define an enumeration that returns a state describing a single field
of the board:

In [ ]:
class Field(Enum):
    EMPTY = "\N{Open Box}"
    LIGHT = "\N{Medium White Circle}"
    DARK = "\N{Medium Black Circle}"

In [ ]:
print("Fields:           ", list(Field))
print("Empty field:      ", Field.DARK)
print("Light field:      ", Field.LIGHT)
print("Dark field:       ", Field.DARK)
print("Light field value:", Field.LIGHT.value)
print("Light field name: ", Field.LIGHT.name)

## Mini workshop

Define a function `is_occupied(f: Field) -> bool`,
which returns `True` if and only if `field` is not empty.

In [ ]:
def is_occupied(f: Field) -> bool:
    return f is not Field.EMPTY

In [ ]:
assert is_occupied(Field.DARK)
assert is_occupied(Field.LIGHT)
assert not is_occupied(Field.EMPTY)

## Mini workshop

Define an enumeration `Player` which describes the players in analogy with the enumeration `Field`.
Print out the possible values as well as value and name for a single enum value.

In [ ]:
class Player(Enum):
    LIGHT = 0
    DARK = 1

In [ ]:
print("Players:           ", list(Player))
print("Light player:      ", Player.LIGHT)
print("Dark player:       ", Player.DARK)
print("Light player value:", Player.LIGHT.value)
print("Light player name: ", Player.LIGHT.name)

## Mini workshop

Define a function
`is_field_owned_by_opponent(p: Player, f: Field) -> bool`,
which returns `True` if and only if the token on square `f` belongs to the opponen, i.e. if `f` is occupied and the token on `f` does not have the same color as the player.

Verify that your implementation satisfies the specified assertions.

Implement a corresponding method `is_field_owned_by_player()`.

In [ ]:
def is_field_owned_by_opponent(p: Player, f: Field) -> bool:
    if p is Player.DARK:
        return f is Field.LIGHT
    else:
        return f is Field.DARK

In [ ]:
assert is_field_owned_by_opponent(Player.LIGHT, Field.DARK)
assert is_field_owned_by_opponent(Player.DARK, Field.LIGHT)
assert not is_field_owned_by_opponent(Player.DARK, Field.DARK)
assert not is_field_owned_by_opponent(Player.DARK, Field.EMPTY)
assert not is_field_owned_by_opponent(Player.LIGHT, Field.LIGHT)
assert not is_field_owned_by_opponent(Player.LIGHT, Field.EMPTY)

In [ ]:
def is_field_owned_by_player(p: Player, f: Field) -> bool:
    if p is Player.DARK:
        return f is Field.DARK
    else:
        return f is Field.LIGHT

In [ ]:
assert is_field_owned_by_player(Player.LIGHT, Field.LIGHT)
assert is_field_owned_by_player(Player.DARK, Field.DARK)
assert not is_field_owned_by_player(Player.DARK, Field.LIGHT)
assert not is_field_owned_by_player(Player.DARK, Field.EMPTY)
assert not is_field_owned_by_player(Player.LIGHT, Field.DARK)
assert not is_field_owned_by_player(Player.LIGHT, Field.EMPTY)

We want to go to the individual squares of an 8x8 playing field using a
two-dimensional index value: `board[0, 0]` stands for the left one
top panel, `board[0, 1]` for the panel to the right, etc.

This is easy if we take a two-dimensional array as the underlying
data structure, e.g. NumPy's `ndarray`. In this case
let's just delegate `__getitem__()` and `__setitem__()` to the
corresponding methods of the NumPy array.

We initialize the middle 4 fields of the playing field with diagonal
arranged stones.

In [ ]:
@dataclass()
class NumPyBoard:
    _fields: np.array = field(default_factory=lambda: np.array([[Field.EMPTY] * 8] * 8))

    def __post_init__(self):
        self[3, 3] = Field.DARK
        self[3, 4] = Field.LIGHT
        self[4, 3] = Field.LIGHT
        self[4, 4] = Field.DARK

    def __getitem__(self, item) -> Union[Field, np.ndarray]:
        return self._fields[item]

    def __setitem__(self, key, value) -> None:
        self._fields[key] = value

    def __iter__(self):
        return iter(self._fields)

    def __repr__(self) -> str:
        name = type(self).__name__
        return f"{name}({reprlib.repr(self._fields)})"

    def __str__(self) -> str:
        result = ""
        prefix = "|"
        for row_index in range(8):
            result += prefix
            for col in range(8):
                result += str(self[row_index, col].value) + "|"
            prefix = "\n|"
        return result

In [ ]:
npb = NumPyBoard()
npb[0, 0] = Field.DARK
npb[0, 1] = Field.LIGHT
print(npb)

Because the index is passed to the NumPy array,
powerful access variants, such as slicing, are also available:

In [ ]:
npb[0, 2:] = Field.DARK
npb[1:3, 1:3] = Field.LIGHT
print(npb)

## Mini workshop Compute Linear Index

If we want to use a Python list as storage for the playing field,
we need to convert two-dimensional index values into a one-dimensional index.

We do this by calculating the value `r * 8 + c` for a position `(r, c)`.
In the event that one of the indices is not `>= 0` and `< 8`, a
appropriate exception are raised.

Implement a function `compute_linear_index()` to do this task
takes over.

In [ ]:
def compute_linear_index(index):
    """
    Compute the linear index for the given (possibly complex) index.

    If `index` is already a linear index or sliced it is returned unchanged.
    If it is a pair of numbers, each between 0 and 7, the corresponding
    linear index into an 8x8 grid is computed, using the usual conventions
    for multidimensional indexing, i.e., the first index is the row, the
    second index the column.

    (For a real application this function should probably be extended to
    convert 2-dimensional slices into 1-dimensional slices as well, but that
    is too complex for this example.)

    :param index: A linear index, a pair of indices or a linear slice
    :return: A linear index or slice
    """
    try:
        row, column = index
        if 0 <= row <= 7:
            raise ValueError(f"Row index {row} out of range.")
        if 0 <= column <= 7:
            raise ValueError("Column index out of range.")
        return row * 8 + column
    except TypeError:
        return index

## Mini workshop

Implement a class `Board` representing an 8x8 Othellite board
that stores the state of the board in a list.
You can refer to the `NumPyBoard` class for implementation hints, and you can use `compute_linear_index` to calculate index values.

In [ ]:
@dataclass(repr=False)
class Board:
    _fields: list[Field] = field(default_factory=lambda: [Field.EMPTY] * 64)

    def __post_init__(self):
        self[3, 3] = Field.DARK
        self[3, 4] = Field.LIGHT
        self[4, 3] = Field.LIGHT
        self[4, 4] = Field.DARK

    def __getitem__(self, index) -> Field:
        return self._fields[compute_linear_index(index)]

    def __setitem__(self, index, value):
        self._fields[compute_linear_index(index)] = value

    def __iter__(self):
        return iter(self._fields)

    def __repr__(self) -> str:
        name = type(self).__name__
        return f"{name}({reprlib.repr(self._fields)})"

    def __str__(self) -> str:
        result = ""
        prefix = "|"
        for row in range(8):
            result += prefix
            for col in range(8):
                result += str(self[row, col].value) + "|"
            prefix = "\n|"
        return result

In [ ]:
board = Board()
board[0, 0] = Field.DARK
board[0, 2] = Field.DARK
board[0, 1] = Field.LIGHT
print(str(board))
board[:3]

We define another enumeration for "directions". The
Keys of the enumeration should be abbreviations for the compass directions (N,
NE, E, ...). The value for movement in this direction
should be the "offset" to move in that direction:

- `(-1, 0)` for northbound movement,
- `(0, 1)` for eastbound movement,
- `(1, 0)` for a southbound movement,
- `(0, -1)` for westbound movement,
- `(-1, 1)` for a north-east movement,
- `(1, 1)` for moving south-east,
- `(-1, -1)` for moving north-west,
- `(1, -1` for moving south-west.

<!--
We define yet another enumeration, this time for directions.
The keys of this enumeration should be the compass directions (N, NE, E, ...),
the values should be the (x, y) offset to move to this field in a
right-handed coordinate system, i.e.,
- `(-1, 0)` for moving north,
- `(0, 1)` for moving east,
- `(1, 0)` for moving south,
- `(0, -1)` for moving west,
- `(-1, 1)` for moving north-east,
- `(1, 1)` for moving south-east,
- `(-1, -1)` for moving north-west,
- `(1, -1` for moving south-west.
-->

## Mini workshop

Define an enumeration `Directions` as just described.

In [ ]:
class DirectionV0(Enum):
    N = (-1, 0)
    NE = (-1, 1)
    E = (0, 1)
    SE = (1, 1)
    S = (1, 0)
    SW = (1, -1)
    W = (0, -1)
    NW = (-1, -1)

It is possible with enumerations to use several keys for the same value
to define (i.e. to give synonyms, so to speak).

```python
class Synonyms(enum):
    DRINK = 0
    BEVERAGE = 0
    FOOD = 1
    BIG = 2
    LARGE = 2
    SMALL = 3
```

In [ ]:
print(Synonyms.DRINK is Synonyms.BEVERAGE)
print(Synonyms.DRINK is not Synonyms.FOOD)
print(Synonyms.BIG is Synonyms.LARGE)
print(Synonyms.BIG is not Synonyms.SMALL)

Extend the enumeration `Directions` to include the full
Names of the compass directions (`NORTH`, `NORTH_EAST`, ...).

In [ ]:
class Direction(Enum):
    # Abbreviated names
    N = (-1, 0)
    NE = (-1, 1)
    E = (0, 1)
    SE = (1, 1)
    S = (1, 0)
    SW = (1, -1)
    W = (0, -1)
    NW = (-1, -1)
    # Full names as aliases
    NORTH = (-1, 0)
    NORTH_EAST = (-1, 1)
    EAST = (0, 1)
    SOUTH_EAST = (1, 1)
    SOUTH = (1, 0)
    SOUTH_WEST = (1, -1)
    WEST = (0, -1)
    NORTH_WEST = (-1, -1)

## Mini workshop

Check that the abbreviations and the full names
represent the same values.

In [ ]:
assert Direction.N is Direction.NORTH
assert Direction.NE is Direction.NORTH_EAST
assert Direction.E is Direction.EAST
assert Direction.SE is Direction.SOUTH_EAST
assert Direction.S is Direction.SOUTH
assert Direction.SW is Direction.SOUTH_WEST
assert Direction.W is Direction.WEST
assert Direction.NW is Direction.NORTH_WEST

In [ ]:
print("Direction[0:3]: ", list(Direction)[:3])
print("Direction values:", [d.value for d in Direction])
print("Direction names: ", [d.name for d in Direction])

In [ ]:
Index = tuple[int, int]

## Mini workshop

Write a function `is_valid_index(index: index) -> bool`,
which returns `True` if and only if `index` is a valid index for
an Othellite board. Check if your
Implementation satisfies the given assertions.

In [ ]:
def is_valid_index(index: Index) -> bool:
    row, column = index
    return 0 <= row < 8 and 0 <= column < 8

In [ ]:
assert is_valid_index((0, 7))
assert not is_valid_index((8, 1))
assert not is_valid_index((1, 8))
assert not is_valid_index((6, -1))
assert not is_valid_index((-2, 4))

## Mini workshop

Write a function
`assert_valid_index(index: Index) -> None:` which throws an exception of
type `IndexError` with an appropriate error message if `index`
is not a valid index for an Othellite board. Check that the
function does not throw an exception for correct arguments, but it does for
different error cases.

In [ ]:
def assert_valid_index(index: Index) -> None:
    def assert_component_in_range(value: int, component: str):
        if value < 0 or value >= 8:
            raise IndexError(f"{component} component of index {index} out or range.")

    x, y = index
    assert_component_in_range(x, "First")
    assert_component_in_range(y, "Second")

In [ ]:
assert_valid_index((2, 0))

In [ ]:
try:
    assert_valid_index((-1, 3))
except IndexError as err:
    print(err)

In [ ]:
try:
    assert_valid_index((1, 8))
except IndexError as err:
    print(err)

Write a function
`next_index_in_direction(index: Index, direction: Direction)`,
which returns two values if `index` is a valid index:
- the next index in direction `direction` and
- `True` or `False` depending on whether the next index is valid or not.
If `index` is not a valid index, an exception of type
`IndexError` is thrown.

*Note:* You can calculate the values to be added to the components of `index`
using `d_row, d_column = direction.value`.

Verify that your implementation satisfies the specified assertions.

In [ ]:
def next_index_in_direction(index: Index, direction: Direction):
    assert_valid_index(index)
    row, column = index
    d_row, d_column = direction.value
    next_index = (row + d_row, column + d_column)
    return next_index, is_valid_index(next_index)

In [ ]:
assert next_index_in_direction((0, 0), Direction.N) == ((-1, 0), False)
assert next_index_in_direction((0, 0), Direction.NE) == ((-1, 1), False)
assert next_index_in_direction((0, 0), Direction.E) == ((0, 1), True)
assert next_index_in_direction((0, 0), Direction.SE) == ((1, 1), True)
assert next_index_in_direction((0, 0), Direction.S) == ((1, 0), True)
assert next_index_in_direction((0, 0), Direction.SW) == ((1, -1), False)
assert next_index_in_direction((0, 0), Direction.W) == ((0, -1), False)
assert next_index_in_direction((0, 0), Direction.NW) == ((-1, -1), False)

In [ ]:
assert next_index_in_direction((4, 5), Direction.N) == ((3, 5), True)
assert next_index_in_direction((4, 5), Direction.NE) == ((3, 6), True)
assert next_index_in_direction((4, 5), Direction.E) == ((4, 6), True)
assert next_index_in_direction((4, 5), Direction.SE) == ((5, 6), True)
assert next_index_in_direction((4, 5), Direction.S) == ((5, 5), True)
assert next_index_in_direction((4, 5), Direction.SW) == ((5, 4), True)
assert next_index_in_direction((4, 5), Direction.W) == ((4, 4), True)
assert next_index_in_direction((4, 5), Direction.NW) == ((3, 4), True)

In [ ]:
assert next_index_in_direction((7, 7), Direction.N) == ((6, 7), True)
assert next_index_in_direction((7, 7), Direction.NE) == ((6, 8), False)
assert next_index_in_direction((7, 7), Direction.E) == ((7, 8), False)
assert next_index_in_direction((7, 7), Direction.SE) == ((8, 8), False)
assert next_index_in_direction((7, 7), Direction.S) == ((8, 7), False)
assert next_index_in_direction((7, 7), Direction.SW) == ((8, 6), False)
assert next_index_in_direction((7, 7), Direction.W) == ((7, 6), True)
assert next_index_in_direction((7, 7), Direction.NW) == ((6, 6), True)

In [ ]:
try:
    next_index_in_direction((7, 8), Direction.S)
except IndexError as err:
    print(err)

We want to extend our `Board` class with a method

```
_fields_flipped_in_direction(self, p: Player, index: Index,
                             d: Direction) -> set[Index]
```

which returns the indices of all fields that the player
can flip starting from field `index` and moving in direction `d`.

In [ ]:
# This is very specific to the use in `Board`...
def _find_rightmost(seq: Sequence):
    for i in range(len(seq) - 1, -1, -1):
        if seq[i]:
            return i
    return 0

In [ ]:
assert _find_rightmost([True, False, True, False]) == 2
assert _find_rightmost([False, False, False]) == 0

In [ ]:
@dataclass(repr=False)
class Board:
    _fields: list[Field] = field(default_factory=lambda: [Field.EMPTY] * 64)

    def __post_init__(self):
        self[3, 3] = Field.DARK
        self[3, 4] = Field.LIGHT
        self[4, 3] = Field.LIGHT
        self[4, 4] = Field.DARK

    def __getitem__(self, index) -> Field:
        return self._fields[compute_linear_index(index)]

    def __setitem__(self, index, value):
        self._fields[compute_linear_index(index)] = value

    def __iter__(self):
        return iter(self._fields)

    def __repr__(self) -> str:
        name = type(self).__name__
        return f"{name}({reprlib.repr(self._fields)})"

    def __str__(self) -> str:
        result = ""
        prefix = "|"
        for row in range(8):
            result += prefix
            for col in range(8):
                result += str(self[row, col].value) + "|"
            prefix = "\n|"
        return result

    def _indices_that_can_be_flipped(
        self, player: Player, occupied_indices: list[Index]
    ) -> set[Index]:
        is_self_owned_field = [
            is_field_owned_by_player(player, self[index]) for index in occupied_indices
        ]
        stop_index = _find_rightmost(is_self_owned_field)
        return set(
            index
            for index in occupied_indices[:stop_index]
            if is_field_owned_by_opponent(player, self[index])
        )

    def _indices_to_flip_in_direction(
        self, p: Player, index: Index, d: Direction
    ) -> set[Index]:
        next_index, is_valid = next_index_in_direction(index, d)
        occupied_indices = []
        while is_valid:
            if not is_occupied(self[next_index]):
                break
            occupied_indices.append(next_index)
            next_index, is_valid = next_index_in_direction(next_index, d)
        return self._indices_that_can_be_flipped(p, occupied_indices)

In [ ]:
def assert_flips_direction(player, index, directions):
    board = Board()
    board[index] = Field.LIGHT if player == Player.LIGHT else Field.DARK
    print(f"Player {player} at {index} flips {directions}.")
    print(board)

    for d in set(Direction) - directions:
        # noinspection PyProtectedMember
        result = board._indices_to_flip_in_direction(player, index, d)
        assert result == set(), f"Fields flipped in {d} are not empty."

    for d in directions:
        # noinspection PyProtectedMember
        result = board._indices_to_flip_in_direction(player, index, d)
        assert result != set(), f"Fields flipped in {d} are empty."

In [ ]:
assert_flips_direction(Player.LIGHT, (0, 0), set())
assert_flips_direction(Player.LIGHT, (2, 3), {Direction.S})
assert_flips_direction(Player.LIGHT, (3, 2), {Direction.E})
assert_flips_direction(Player.LIGHT, (4, 5), {Direction.W})
assert_flips_direction(Player.LIGHT, (5, 2), set())
assert_flips_direction(Player.LIGHT, (5, 3), set())
assert_flips_direction(Player.LIGHT, (5, 4), {Direction.N})
assert_flips_direction(Player.LIGHT, (6, 4), set())

In [ ]:
assert_flips_direction(Player.DARK, (0, 0), set())
assert_flips_direction(Player.DARK, (2, 4), {Direction.S})
assert_flips_direction(Player.DARK, (3, 5), {Direction.W})
assert_flips_direction(Player.DARK, (4, 2), {Direction.E})
assert_flips_direction(Player.DARK, (5, 2), set())
assert_flips_direction(Player.DARK, (5, 3), {Direction.N})
assert_flips_direction(Player.DARK, (5, 4), set())
assert_flips_direction(Player.DARK, (6, 4), set())

## Mini workshop

Add a method to the `Board` class
`is_valid_move(self, player: Player, index: Index) -> bool`
which returns `True` if and only if `index` is a valid index for
the player's next move.

An index is valid if
- It is not already occupied,
- At least one of the opponent's stones is turned over by the move

In [ ]:
@dataclass(repr=False)
class Board:
    _fields: list[Field] = field(default_factory=lambda: [Field.EMPTY] * 64)

    def __post_init__(self):
        self[3, 3] = Field.DARK
        self[3, 4] = Field.LIGHT
        self[4, 3] = Field.LIGHT
        self[4, 4] = Field.DARK

    def __getitem__(self, index) -> Field:
        return self._fields[compute_linear_index(index)]

    def __setitem__(self, index, value):
        self._fields[compute_linear_index(index)] = value

    def __iter__(self):
        return iter(self._fields)

    def __repr__(self) -> str:
        name = type(self).__name__
        return f"{name}({reprlib.repr(self._fields)})"

    def __str__(self) -> str:
        result = ""
        prefix = "|"
        for row in range(8):
            result += prefix
            for col in range(8):
                result += str(self[row, col].value) + "|"
            prefix = "\n|"
        return result

    def _indices_that_can_be_flipped(
        self, player: Player, occupied_indices: list[Index]
    ) -> set[Index]:
        is_self_owned_field = [
            is_field_owned_by_player(player, self[index]) for index in occupied_indices
        ]
        stop_index = _find_rightmost(is_self_owned_field)
        return set(
            index
            for index in occupied_indices[:stop_index]
            if is_field_owned_by_opponent(player, self[index])
        )

    def _indices_to_flip_in_direction(
        self, p: Player, index: Index, d: Direction
    ) -> set[Index]:
        next_index, is_valid = next_index_in_direction(index, d)
        occupied_indices = []
        while is_valid:
            if not is_occupied(self[next_index]):
                break
            occupied_indices.append(next_index)
            next_index, is_valid = next_index_in_direction(next_index, d)
        return self._indices_that_can_be_flipped(p, occupied_indices)

    def is_valid_move(self, player: Player, index: Index) -> bool:
        if is_occupied(self[index]):
            return False
        for d in Direction:
            if self._indices_to_flip_in_direction(player, index, d):
                return True
        return False

In [ ]:
def assert_valid_moves(player, valid_indices):
    board = Board()
    for row in range(8):
        for col in range(8):
            index = (row, col)
            result = board.is_valid_move(player, index)
            if index in valid_indices:
                assert result, f"Index {index} should be valid."
            else:
                assert not result, f"Index {index} is valid?"

In [ ]:
print(Board())

In [ ]:
valid_moves_for_light_player = {(2, 3), (3, 2), (4, 5), (5, 4)}
assert_valid_moves(Player.LIGHT, valid_moves_for_light_player)

In [ ]:
valid_move_for_dark_player = {(2, 4), (3, 5), (4, 2), (5, 3)}
assert_valid_moves(Player.DARK, valid_move_for_dark_player)

## Mini workshop

Add a method to the `Board` class
`find_valid_moves(self, player: Player) -> set[Index]`,
which returns all possible moves for the player `player`.

*Note:* You can easily achieve this using *Generate and Test*:
Generate all possible moves and then test for each move whether
he is possible.

In [ ]:
@dataclass(repr=False)
class Board:
    _fields: list[Field] = field(default_factory=lambda: [Field.EMPTY] * 64)

    def __post_init__(self):
        self[3, 3] = Field.DARK
        self[3, 4] = Field.LIGHT
        self[4, 3] = Field.LIGHT
        self[4, 4] = Field.DARK

    def __getitem__(self, index) -> Field:
        return self._fields[compute_linear_index(index)]

    def __setitem__(self, index, value):
        self._fields[compute_linear_index(index)] = value

    def __iter__(self):
        return iter(self._fields)

    def __repr__(self) -> str:
        name = type(self).__name__
        return f"{name}({reprlib.repr(self._fields)})"

    def __str__(self) -> str:
        result = ""
        prefix = "|"
        for row in range(8):
            result += prefix
            for col in range(8):
                result += str(self[row, col].value) + "|"
            prefix = "\n|"
        return result

    def _indices_that_can_be_flipped(
        self, player: Player, occupied_indices: list[Index]
    ) -> set[Index]:
        is_self_owned_field = [
            is_field_owned_by_player(player, self[index]) for index in occupied_indices
        ]
        stop_index = _find_rightmost(is_self_owned_field)
        return set(
            index
            for index in occupied_indices[:stop_index]
            if is_field_owned_by_opponent(player, self[index])
        )

    def _indices_to_flip_in_direction(
        self, p: Player, index: Index, d: Direction
    ) -> set[Index]:
        next_index, is_valid = next_index_in_direction(index, d)
        occupied_indices = []
        while is_valid:
            if not is_occupied(self[next_index]):
                break
            occupied_indices.append(next_index)
            next_index, is_valid = next_index_in_direction(next_index, d)
        return self._indices_that_can_be_flipped(p, occupied_indices)

    def is_valid_move(self, player: Player, index: Index) -> bool:
        if is_occupied(self[index]):
            return False
        for d in Direction:
            if self._indices_to_flip_in_direction(player, index, d):
                return True
        return False

    def find_valid_moves(self, player: Player) -> set[Index]:
        result = set()
        for row in range(8):
            for column in range(8):
                move = (row, column)
                if self.is_valid_move(player, move):
                    result.add(move)
        return result

In [ ]:
board = Board()
assert board.find_valid_moves(Player.LIGHT) == valid_moves_for_light_player
assert board.find_valid_moves(Player.DARK) == valid_move_for_dark_player

## Mini workshop

Write a method
`play_move(self, player: Player, index: Index) -> None`,
which executes the move from `player` to the field `index`.
If the move is invalid, an exception should be thrown.
If the move is valid, all fields flipped by the move should
be set to the correct value.

To do this, write appropriate helper methods so that `play_move` does not
have too many responsibilities.

In [ ]:
@dataclass(repr=False)
class Board:
    _fields: list[Field] = field(default_factory=lambda: [Field.EMPTY] * 64)

    def __post_init__(self):
        self[3, 3] = Field.DARK
        self[3, 4] = Field.LIGHT
        self[4, 3] = Field.LIGHT
        self[4, 4] = Field.DARK

    def __getitem__(self, index) -> Field:
        return self._fields[compute_linear_index(index)]

    def __setitem__(self, index, value):
        self._fields[compute_linear_index(index)] = value

    def __iter__(self):
        return iter(self._fields)

    def __repr__(self) -> str:
        name = type(self).__name__
        return f"{name}({reprlib.repr(self._fields)})"

    def __str__(self) -> str:
        result = ""
        prefix = "|"
        for row in range(8):
            result += prefix
            for col in range(8):
                result += str(self[row, col].value) + "|"
            prefix = "\n|"
        return result

    def _indices_that_can_be_flipped(
        self, player: Player, occupied_indices: list[Index]
    ) -> set[Index]:
        is_self_owned_field = [
            is_field_owned_by_player(player, self[index]) for index in occupied_indices
        ]
        stop_index = _find_rightmost(is_self_owned_field)
        return set(
            index
            for index in occupied_indices[:stop_index]
            if is_field_owned_by_opponent(player, self[index])
        )

    def _indices_to_flip_in_direction(
        self, p: Player, index: Index, d: Direction
    ) -> set[Index]:
        next_index, is_valid = next_index_in_direction(index, d)
        occupied_indices = []
        while is_valid:
            if not is_occupied(self[next_index]):
                break
            occupied_indices.append(next_index)
            next_index, is_valid = next_index_in_direction(next_index, d)
        return self._indices_that_can_be_flipped(p, occupied_indices)

    def is_valid_move(self, player: Player, index: Index) -> bool:
        if is_occupied(self[index]):
            return False
        for d in Direction:
            if self._indices_to_flip_in_direction(player, index, d):
                return True
        return False

    def find_valid_moves(self, player: Player) -> set[Index]:
        result = set()
        for row in range(8):
            for column in range(8):
                move = (row, column)
                if self.is_valid_move(player, move):
                    result.add(move)
        return result

    def play_move(self, player: Player, index: Index) -> None:
        if self.is_valid_move(player, index):
            field = Field.LIGHT if player is Player.LIGHT else Field.DARK
            self[index] = field
            flipped_indices = self.find_flipped_indices(player, index)
            self.flip_indices(field, flipped_indices)
        else:
            raise ValueError(f"{index} is not a valid move")

    def find_flipped_indices(self, player: Player, index: Index) -> set[Index]:
        result = set()
        for d in Direction:
            result |= self._indices_to_flip_in_direction(player, index, d)
        return result

    def flip_indices(self, field: Field, flipped_indices: set[Index]) -> None:
        for index in flipped_indices:
            self[index] = field

In [ ]:
def play_some_random_moves(n: int = 10):
    board = Board()
    player = Player.DARK
    for _ in range(n):
        print(board)
        moves = board.find_valid_moves(player)
        if moves:
            move = sample(list(moves), 1)[0]
            print(f"Player {player} plays {move}.")
            board.play_move(player, move)
        else:
            print(f"{player} has no more valid moves.")
        player = Player.LIGHT if player == Player.DARK else Player.DARK
    print(board)

In [ ]:
play_some_random_moves()